### Import important libraries

In [5]:
import pandas as pd
import numpy as np

### Merge data and output

In [6]:
full=pd.merge(pd.read_csv("family_data.csv"),pd.read_csv("sample_submission.csv"))

### Reusable Functions

In [7]:
def prefCost(people,choice=-1):
    consolation=[[0,0],[50,0],[50,9],[100,9],[200,9],[200,18],[300,18],[300,36],[400,36],[500,36],[500,36+398]]
    return (consolation[choice][0]+consolation[choice][1]*people*1.0) 

def accCost(full):
    segregated=(full.groupby(by=['assigned_day']).sum())[['n_people']]
    segregated.loc[101]=segregated.loc[100]
    segregated['Cost']=0.0
    segregated=segregated.reset_index('assigned_day')
    if((segregated['n_people']<125).any()):
        return -1
    if((segregated['n_people']>300).any()):
        return -1
    accounting_cost=0
    for day in range(100):
        diff=abs(segregated['n_people'][day]-segregated['n_people'][day+1])
        expPart=0.5+(diff/50.0)
        leftPart=(segregated['n_people'][day]-125.0)/400.0
        rightPart=segregated['n_people'][day] ** expPart
        segregated['Cost'][day]=(leftPart*rightPart)
    return segregated['Cost'].sum()

### Initial Assignment

In [8]:
#Assign -1 to all in full
full['assigned_choice']=-1
full['assigned_day']=-1
full['pref_cost']=-1

#Make a table for day-wise grouping
day_table=pd.DataFrame({'Day': range(1,101),'Count':0})
day_table.set_index('Day')


for day in range(0,100):
    print("Working for day "+str(day+1))
    #print("=========================")
    day_table.loc[day]['Count']=0
    for family in range(5000):
        #rint("Working on "+str(family))
        if(day_table.loc[day]['Count']>=208):
            print("Total for day "+str(day+1)+"="+str(day_table.loc[day]['Count']))
            break
        if(full['assigned_day'][family]>-1):
            continue
        full['assigned_day'][family]=day+1
        day_table['Count'][day]+=full['n_people'][family]
        #print("Adding family_id "+str(family))
        for choice in range(0,11):
            if(choice==10):
                full['assigned_choice'][family]=choice
                break
            if(full['choice_'+str(choice)][family]==day):
                full['assigned_choice'][family]=choice
                break
        
        full['pref_cost'][family]=prefCost(full['n_people'][family],full['assigned_choice'][family])

Working for day 1
Total for day 1=209
Working for day 2
Total for day 2=210
Working for day 3
Total for day 3=210
Working for day 4
Total for day 4=209
Working for day 5
Total for day 5=210
Working for day 6
Total for day 6=208
Working for day 7
Total for day 7=208
Working for day 8
Total for day 8=213
Working for day 9
Total for day 9=211
Working for day 10
Total for day 10=214
Working for day 11
Total for day 11=213
Working for day 12
Total for day 12=210
Working for day 13
Total for day 13=212
Working for day 14
Total for day 14=210
Working for day 15
Total for day 15=209
Working for day 16
Total for day 16=208
Working for day 17
Total for day 17=211
Working for day 18
Total for day 18=208
Working for day 19
Total for day 19=208
Working for day 20
Total for day 20=209
Working for day 21
Total for day 21=209
Working for day 22
Total for day 22=211
Working for day 23
Total for day 23=209
Working for day 24
Total for day 24=208
Working for day 25
Total for day 25=209
Working for day 26

### Verify Total Count should be correct in assignment

In [9]:
day_table['Count'].sum()

21003

In [10]:
full['n_people'].sum()

21003

### Check initial cost

In [11]:
print("Acc Cost="+str(accCost(full)))
print("Pref Cost="+str(full['pref_cost'].sum()))
print("score="+str(accCost(full)+full['pref_cost'].sum()))

Acc Cost=378.6761161291622
Pref Cost=10574644
score=10575022.67611613


C:\Users\singgai\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Start Optimization by looping over choices now

In [12]:
full_bk=full.copy()

In [24]:
full=full_bk.copy()

In [13]:
def optimize(full, targetted_ids_list):
    #print("Input Cost="+str(accCost(full)+full['pref_cost'].sum()))
    print("score="+str(accCost(full)+full['pref_cost'].sum()))
    for family in targetted_ids_list:
        full_test=full
        curr_acc_cost=accCost(full)
        curr_pref_cost=full['pref_cost'].sum()
        curr_cost=curr_acc_cost+curr_pref_cost
        curr_choice=full_test['assigned_choice'][family]
    
        #print("Family Id:"+str(family)+" Cost="+str(curr_cost)+"(Pref="+str(full['pref_cost'][family])+")"+" Day="+str(full['assigned_day'][family])+" Choice="+str(full['assigned_choice'][family]))
        
        #Assign optimized choice and cost same as current one
        optimized_choice=full_test['assigned_choice'][family]
        optimized_cost=curr_cost
        for choice in range(10):
            full_test['assigned_day'][family]=full_test['choice_'+str(choice)][family]
            full_test['assigned_choice'][family]=choice
            full_test['pref_cost'][family]=prefCost(full_test['n_people'][family],choice)
            calculated_acc_cost=accCost(full_test)
            if(calculated_acc_cost==-1):
                #print("Leaving Choice "+str(choice))
                continue
            calculated_cost=calculated_acc_cost+full_test['pref_cost'].sum()
            #print("Choice="+str(choice)+" Calculated Cost="+str(calculated_cost)+" (Pref="+str(full_test['pref_cost'][family])+")")
            if(calculated_cost<optimized_cost):
                optimized_cost=calculated_cost
                optimized_choice=choice
        if(curr_choice!=optimized_choice):
            full['assigned_day'][family]=full['choice_'+str(optimized_choice)][family]
            full['assigned_choice'][family]=optimized_choice
            full['pref_cost'][family]=prefCost(full['n_people'][family],full['assigned_choice'][family])
        reduction=curr_cost-optimized_cost
        #print("NEW: Family Id:"+str(family)+" Cost="+str(optimized_cost)+"(Pref="+str(full['pref_cost'][family])+")"+" Day="+str(full['assigned_day'][family])+" Choice="+str(full['assigned_choice'][family])+" Reduction="+str(reduction))
        #print("=========================================================================")
    #print("Output Cost="+str(accCost(full)+full['pref_cost'].sum()))
    print("score="+str(accCost(full)+full['pref_cost'].sum()))

### Targetting all records with Choice=10

In [14]:
pd.options.display.max_rows=100
pd.options.display.max_columns=100
full.groupby(by=['assigned_choice','n_people']).describe()[['pref_cost']]

pref_cost                                       \
                             count    mean  std     min     25%     50%   
assigned_choice n_people                                                  
0               2              7.0     0.0  0.0     0.0     0.0     0.0   
                3              9.0     0.0  0.0     0.0     0.0     0.0   
                4             15.0     0.0  0.0     0.0     0.0     0.0   
                5             11.0     0.0  0.0     0.0     0.0     0.0   
                6              5.0     0.0  0.0     0.0     0.0     0.0   
                7              6.0     0.0  0.0     0.0     0.0     0.0   
                8              3.0     0.0  0.0     0.0     0.0     0.0   
1               2             10.0    50.0  0.0    50.0    50.0    50.0   
                3             12.0    50.0  0.0    50.0    50.0    50.0   
                4             15.0    50.0  0.0    50.0    50.0    50.0   
                5             13.0    50.0  0.0    50.0    50.0    50.0   
                6              6.0    50.0  0.0    50.0    50.0    50.0   
                7              1.0    50.0  NaN    50.0    50.0    50.0   
                8              1.0    50.0  NaN    50.0    50.0    50.0   
2               2              7.0    68.0  0.0    68.0    68.0    68.0   
                3             12.0    77.0  0.0    77.0    77.0    77.0   
                4             12.0    86.0  0.0    86.0    86.0    86.0   
                5              3.0    95.0  0.0    95.0    95.0    95.0   
                6              4.0   104.0  0.0   104.0   104.0   104.0   
                7              1.0   113.0  NaN   113.0   113.0   113.0   
                8              1.0   122.0  NaN   122.0   122.0   122.0   
3               2              5.0   118.0  0.0   118.0   118.0   118.0   
                3              8.0   127.0  0.0   127.0   127.0   127.0   
                4             22.0   136.0  0.0   136.0   136.0   136.0   
                5              7.0   145.0  0.0   145.0   145.0   145.0   
                6              3.0   154.0  0.0   154.0   154.0   154.0   
                7              1.0   163.0  NaN   163.0   163.0   163.0   
                8              1.0   172.0  NaN   172.0   172.0   172.0   
4               2             10.0   218.0  0.0   218.0   218.0   218.0   
                3             12.0   227.0  0.0   227.0   227.0   227.0   
                4             20.0   236.0  0.0   236.0   236.0   236.0   
                5             10.0   245.0  0.0   245.0   245.0   245.0   
                6              6.0   254.0  0.0   254.0   254.0   254.0   
                7              2.0   263.0  0.0   263.0   263.0   263.0   
                8              1.0   272.0  NaN   272.0   272.0   272.0   
5               2              7.0   236.0  0.0   236.0   236.0   236.0   
                3              6.0   254.0  0.0   254.0   254.0   254.0   
                4             10.0   272.0  0.0   272.0   272.0   272.0   
                5              9.0   290.0  0.0   290.0   290.0   290.0   
                6              3.0   308.0  0.0   308.0   308.0   308.0   
                7              7.0   326.0  0.0   326.0   326.0   326.0   
                8              2.0   344.0  0.0   344.0   344.0   344.0   
6               2              7.0   336.0  0.0   336.0   336.0   336.0   
                3              6.0   354.0  0.0   354.0   354.0   354.0   
                4              9.0   372.0  0.0   372.0   372.0   372.0   
                5              9.0   390.0  0.0   390.0   390.0   390.0   
                6              8.0   408.0  0.0   408.0   408.0   408.0   
                7              6.0   426.0  0.0   426.0   426.0   426.0   
                8              1.0   444.0  NaN   444.0   444.0   444.0   
7               2              5.0   372.0  0.0   372.0   372.0   372.0   
                3              9.0   408.0  0.0   

In [25]:
%%time
#big_families=list(full[(full['assigned_choice']==1)]['family_id'])
list_to_workupon=[]
list_to_revisit=[]
for choiceFactor in range(10,-1,-1):
    for sizeFactor in range(8,1,-1):
        list_to_workupon=list(full[(full['assigned_choice']==choiceFactor)&(full['n_people']==sizeFactor)]['family_id'])
        print("Working upon a fresh list for choice "+str(choiceFactor)+" and family-size "+str(sizeFactor)+". Length="+str(len(list_to_workupon)))
        optimize(full,list_to_workupon)
        list_to_revisit.extend(list_to_workupon)
        print("Working upon revisit list. Length="+str(len(list_to_revisit)))
        optimize(full,list_to_revisit)

Working upon a fresh list for choice 10 and family-size 8. Length=143
score=10575022.67611613


C:\Users\singgai\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


score=10010366.325868648
Working upon revisit list. Length=143
score=10010366.325868648
score=10009802.317281637
Working upon a fresh list for choice 10 and family-size 7. Length=268
score=10009802.317281637
score=9067806.357557666
Working upon revisit list. Length=411
score=9067806.357557666
score=9065588.667704042
Working upon a fresh list for choice 10 and family-size 6. Length=438
score=9065588.667704042
score=7721036.687647297
Working upon revisit list. Length=849
score=7721036.687647297
score=7715126.7678234745
Working upon a fresh list for choice 10 and family-size 5. Length=813
score=7715126.7678234745
score=5624552.010816179
Working upon revisit list. Length=1662
score=5624552.010816179
score=5576137.7384550935
Working upon a fresh list for choice 10 and family-size 4. Length=1301
score=5576137.7384550935
score=3085138.869101444
Working upon revisit list. Length=2963
score=3085138.869101444
score=2940544.219789376
Working upon a fresh list for choice 10 and family-size 3. Leng

KeyboardInterrupt: 

In [26]:
full.groupby(by=['assigned_day']).sum()[['n_people']]

,n_people
assigned_day,
1,299
2,292
3,300
4,300
5,291
6,268
7,248
8,250
9,270


In [23]:
print(list_to_revisit)

[[41, 42, 134, 295, 350, 366, 370, 386, 413, 486, 504, 580, 636, 660, 751, 785, 799, 806, 819, 870, 873, 874, 878, 896, 939, 958, 1081, 1099, 1110, 1120, 1137, 1158, 1176, 1189, 1192, 1268, 1273, 1331, 1353, 1359, 1422, 1436, 1454, 1472, 1477, 1488, 1498, 1529, 1559, 1572, 1597, 1621, 1645, 1648, 1679, 1723, 1738, 1765, 1808, 1809, 1812, 1813, 1822, 1840, 1852, 1995, 2021, 2031, 2068, 2115, 2182, 2198, 2256, 2318, 2347, 2361, 2404, 2456, 2469, 2519, 2606, 2642, 2680, 2698, 2762, 2782, 2819, 2830, 2898, 2899, 2909, 3017, 3033, 3063, 3121, 3122, 3186, 3219, 3272, 3279, 3297, 3306, 3310, 3354, 3437, 3468, 3472, 3491, 3519, 3571, 3595, 3609, 3638, 3662, 3664, 3742, 3769, 3778, 3792, 3795, 3796, 3953, 3956, 3989, 4078, 4139, 4194, 4302, 4337, 4383, 4416, 4509, 4518, 4543, 4561, 4598, 4733, 4759, 4866, 4890, 4931, 4977, 4990]]
